## 13-9. 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.


In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.23.5
1.10.1
0.6.2


### 1) 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [2]:
rating_file_path = './data/ratings.dat'
ratings_cols = ['user_id', 'movies_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, 
                      engine='python', encoding='ISO-8859-1')
original_data_size=len(ratings)
ratings.head()

,user_id,movies_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


3점 이상만 남깁니다.

In [3]:
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size/original_data_size:.2%}')

original_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


ratings 컬럼의 이름을 counts로 바꿉니다.


In [4]:
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

영화 제목을 보기 위해 메타 데이터를 읽어옵니다.

In [6]:
movie_file_path='./data/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols,
                    engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


영화 타이틀을 소문자로 변환

In [7]:
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


MF model을 구성

## 2) 분석해 봅시다.
---

__ratings에 있는 유니크한 영화 개수__

In [8]:
ratings['movies_id'].nunique()

3628

__ratings에 있는 유니크한 사용자 수__

In [9]:
ratings['user_id'].nunique()

6039

__가장 인기 있는 영화 30개(인기순)__

In [10]:
popular_movies=ratings.groupby('movies_id')['user_id'].count()
popular_movies.sort_values(ascending=False).head(30)

movies_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [11]:
top30_movies = popular_movies.sort_values(ascending=False).head(30).reset_index(inplace=False)
top30_movies

,movies_id,user_id
0,2858,3211
1,260,2910
2,1196,2885
3,1210,2716
4,2028,2561
5,589,2509
6,593,2498
7,1198,2473
8,1270,2460
9,2571,2434


## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가

In [12]:
users_file_path = './data/users.dat'
users_cols = ['user_id', 'gender', 'age', 'occupation', 'zip-code']
users = pd.read_csv(users_file_path, sep='::', names=users_cols, 
                      engine='python', encoding='ISO-8859-1')
original_data_size=len(users)
users.head()

,user_id,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


movies['title']에 없는 영화 5개 추가하기

In [13]:
movies[movies['title'].str.contains('fast')]

,movie_id,title,genre
386,390,faster pussycat! kill! kill! (1965),Action|Comedy|Drama
890,902,breakfast at tiffany's (1961),Drama|Romance
1603,1649,"fast, cheap & out of control (1997)",Documentary
1899,1968,"breakfast club, the (1985)",Comedy|Drama
2438,2507,breakfast of champions (1999),Comedy
3141,3210,fast times at ridgemont high (1982),Comedy


분노의질주(The fast and the furious)는 없으니 분노의질주 시리즈를 추가함
1. movies dataFrame에 추가

In [14]:
# 영화 이름 (연도) 추가
my_favorite = ['the fast and the furious i (2001)', 'the fast and the furious ii (2003)',
              'the fast and the furious iii (2006)', 'the fast and the furious iv (2009)',
              'the fast and the furious v (2011)']
my_movie_id = [4001, 4002, 4003, 4004, 4005]
my_movie_genre = ['Adventure|Crime|Thriller']
# movies dataframe에 추가
my_movies = pd.DataFrame({'movie_id':my_movie_id, 'title':my_favorite, 'genre':['Adventure|Crime|Thriller']*5})
my_movies

,movie_id,title,genre
0,4001,the fast and the furious i (2001),Adventure|Crime|Thriller
1,4002,the fast and the furious ii (2003),Adventure|Crime|Thriller
2,4003,the fast and the furious iii (2006),Adventure|Crime|Thriller
3,4004,the fast and the furious iv (2009),Adventure|Crime|Thriller
4,4005,the fast and the furious v (2011),Adventure|Crime|Thriller


In [15]:
movies_2 = pd.concat([movies, my_movies]).reset_index(drop=True)
movies_2.tail(10)

,movie_id,title,genre
3878,3948,meet the parents (2000),Comedy
3879,3949,requiem for a dream (2000),Drama
3880,3950,tigerland (2000),Drama
3881,3951,two family house (2000),Drama
3882,3952,"contender, the (2000)",Drama|Thriller
3883,4001,the fast and the furious i (2001),Adventure|Crime|Thriller
3884,4002,the fast and the furious ii (2003),Adventure|Crime|Thriller
3885,4003,the fast and the furious iii (2006),Adventure|Crime|Thriller
3886,4004,the fast and the furious iv (2009),Adventure|Crime|Thriller
3887,4005,the fast and the furious v (2011),Adventure|Crime|Thriller


2. users dataFrame에 내 정보 추가

In [16]:
# 'zip-code' 컬럼 삭제
users.drop(labels='zip-code', axis=1, inplace=True)
users.tail(2)

,user_id,gender,age,occupation
6038,6039,F,45,0
6039,6040,M,25,6


In [17]:
my_info = pd.DataFrame({'user_id':6041, 'gender':['F'], 'age':25, 'occupation':15})
my_info

,user_id,gender,age,occupation
0,6041,F,25,15


In [18]:
users_2 = pd.concat([users, my_info]).reset_index(drop=True)
users_2.tail(5)

,user_id,gender,age,occupation
6036,6037,F,45,1
6037,6038,F,56,1
6038,6039,F,45,0
6039,6040,M,25,6
6040,6041,F,25,15


3. ratings dataFrame에 내 정보 추가

In [19]:
ratings.tail(2)

,user_id,movies_id,counts,timestamp
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [20]:
my_ratings = pd.DataFrame({'user_id':[6041]*5, 'movies_id':my_movie_id, 'counts':[5]*5})
my_ratings

,user_id,movies_id,counts
0,6041,4001,5
1,6041,4002,5
2,6041,4003,5
3,6041,4004,5
4,6041,4005,5


In [21]:
ratings_2 = pd.concat([ratings, my_ratings])
ratings_2.tail(10)

,user_id,movies_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,6041,4001,5,NaN
1,6041,4002,5,NaN
2,6041,4003,5,NaN
3,6041,4004,5,NaN
4,6041,4005,5,NaN


In [22]:
max(ratings.movies_id)

3952

In [26]:
ratings_2.tail(2)

,user_id,movies_id,counts,timestamp
3,6041,4004,5,NaN
4,6041,4005,5,NaN


In [27]:
# 고유 유저, 영화아이디 찾아내는 코드
user_unique = ratings_2['user_id'].unique()
movie_unique = movies_2['movie_id'].unique()

# 유저, 영화에 새로운 indexing 할당하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [28]:
index_to_movie = {v:k for k,v in movie_to_idx.items()}

In [29]:
movie_to_idx

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44,
 46: 45,
 47: 46,
 48: 47,
 49: 48,
 50: 49,
 51: 50,
 52: 51,
 53: 52,
 54: 53,
 55: 54,
 56: 55,
 57: 56,
 58: 57,
 59: 58,
 60: 59,
 61: 60,
 62: 61,
 63: 62,
 64: 63,
 65: 64,
 66: 65,
 67: 66,
 68: 67,
 69: 68,
 70: 69,
 71: 70,
 72: 71,
 73: 72,
 74: 73,
 75: 74,
 76: 75,
 77: 76,
 78: 77,
 79: 78,
 80: 79,
 81: 80,
 82: 81,
 83: 82,
 84: 83,
 85: 84,
 86: 85,
 87: 86,
 88: 87,
 89: 88,
 90: 89,
 92: 90,
 93: 91,
 94: 92,
 95: 93,
 96: 94,
 97: 95,
 98: 96,
 99: 97,
 100: 98,
 101: 99,
 102: 100,
 103: 101,
 104: 102,
 105: 103,
 106: 104,
 107: 105,
 108: 106,
 109: 107,
 110: 108,
 111: 109,
 112: 1

In [30]:
len(user_to_idx)

6040

In [31]:
len(movie_to_idx)

3888

새로 추가한 내 아이디와 영화 인덱스 확인

In [32]:
user_to_idx[6041]

6039

In [33]:
movie_to_idx[4001]

3883

In [34]:
movie_to_idx[4005]

3887

In [35]:
len(ratings_2)

836483

* 위에서 enumerate로 user_id와 영화 인덱스를 새로 부여했으니, 얘를 실제 데이터인 ratings_2에 업데이트를 해줘야 함.
* user_to_idx.get을 통해 인덱싱값 Series를 구함
* 정상적으로 인덱싱되지 않은 row는 인덱스가 NaN이 되므로 dropna()로 삭제

In [36]:
# user_id 인덱스 업데이트
temp_user_data = ratings_2['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings_2):
    print('user_id column indexing OK!!')
    ratings_2['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')
    
# movie_id 인덱스 업데이트
temp_movie_data = ratings_2['movies_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings_2):
    print('movies column indexing OK!!')
    ratings_2['movies_id'] = temp_movie_data
else:
    print('movies column indexing Fail!!')
ratings_2

user_id column indexing OK!!
movies column indexing OK!!


,user_id,movies_id,counts,timestamp
0,0,1176,5,978300760.0
1,0,655,3,978302109.0
2,0,902,3,978301968.0
3,0,3339,4,978300275.0
4,0,2286,5,978824291.0
...,...,...,...,...
0,6039,3883,5,NaN
1,6039,3884,5,NaN
2,6039,3885,5,NaN
3,6039,3886,5,NaN


## 4) CSR matrix를 직접 만들어 봅시다.

In [37]:
num_user = ratings_2['user_id'].nunique() # 6040
num_movies = movies_2['movie_id'].nunique() # 3888

In [38]:
from scipy.sparse import csr_matrix

csr_data = csr_matrix((ratings_2.counts, (ratings_2.user_id, ratings_2.movies_id)),
                     shape=(num_user, num_movies))

In [39]:
# csr_data 내부의 원소들 숫자 크기 키워보기
csr_data2 = (csr_data*40).astype('double')

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [40]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [41]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False,
                                  iterations=30, dtype=np.float32)

* als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

In [42]:
# 모델 훈련
als_model.fit(csr_data2)

  0%|          | 0/30 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

__내가 넣은 user_id(6039)와 영화 벡터(3883, 3884, 3885, 3886, 3887)를 곱하면(내적) 몇이 나오는지 확인: 1에 가까울수록 영화에 대한 선호도가 높다는 의미__

In [50]:
user1 = user_to_idx[6041]
my_favorite_id = [4001, 4002, 4003, 4004, 4005]
for i in range(len(my_favorite_id)):
    item = movie_to_idx[my_favorite_id[i]]
    m = movies_2[movies_2['movie_id']==index_to_movie[item]]['title'].iloc[0]
    user1_vector, item_vector = als_model.user_factors[user1], als_model.item_factors[item]
    print(f'영화{i} {m} 에 대한 선호도: {np.dot(user1_vector, item_vector):.3f}')

영화0 the fast and the furious i (2001) 에 대한 선호도: 0.812
영화1 the fast and the furious ii (2003) 에 대한 선호도: 0.812
영화2 the fast and the furious iii (2006) 에 대한 선호도: 0.812
영화3 the fast and the furious iv (2009) 에 대한 선호도: 0.812
영화4 the fast and the furious v (2011) 에 대한 선호도: 0.812


다섯 가지 영화 모두 평점 5점을 부여해서 같은 값이 나왔다.

__그 외 영화에 대한 나의 선호도__

In [52]:
user1 = user_to_idx[6041]
my_favorite_id = [1, 5, 100, 800, 3500]
for i in range(len(my_favorite_id)):
    item = movie_to_idx[my_favorite_id[i]]
    m = movies_2[movies_2['movie_id']==index_to_movie[item]]['title'].iloc[0]
    user1_vector, item_vector = als_model.user_factors[user1], als_model.item_factors[item]
    print(f'영화{i} {m} 에 대한 선호도: {np.dot(user1_vector, item_vector):.3f}')

영화0 toy story (1995) 에 대한 선호도: 0.013
영화1 father of the bride part ii (1995) 에 대한 선호도: -0.254
영화2 city hall (1996) 에 대한 선호도: 0.272
영화3 lone star (1996) 에 대한 선호도: -0.429
영화4 mr. saturday night (1992) 에 대한 선호도: 0.071


In [66]:
my_favorite_id = [1, 5, 100, 800, 3500]
for i in range(len(my_favorite_id)):
    item = movie_to_idx[my_favorite_id[i]]
    index = movies_2[movies_2['movie_id']==index_to_movie[item]]['title']
    print (index)

0    toy story (1995)
Name: title, dtype: object
4    father of the bride part ii (1995)
Name: title, dtype: object
98    city hall (1996)
Name: title, dtype: object
790    lone star (1996)
Name: title, dtype: object
3431    mr. saturday night (1992)
Name: title, dtype: object


In [67]:
movies_2.iloc[[0, 4, 98, 790, 3431]]

,movie_id,title,genre,year
0,1,toy story (1995),Animation|Children's|Comedy,(1995)
4,5,father of the bride part ii (1995),Comedy,(1995)
98,100,city hall (1996),Drama|Thriller,(1996)
790,800,lone star (1996),Drama|Mystery,(1996)
3431,3500,mr. saturday night (1992),Comedy|Drama,(1992)


내가 추가한 영화의 장르는 "Adventure|Crime|Thriller"임. 위 선호도 수치를 보면 Thriller 장르가 포함된 city hall이 가장 높은 값을 갖고(0.272) 그 외 영화는 매우 낮은 선호도를 보임.

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [68]:
# 분노의 질주 I 
movie_id = movie_to_idx[4001]
similar_movie = als_model.similar_items(movie_id, N=10)
similar_movie

(array([3886, 3885, 3883, 3884, 3887, 2588, 3579,  553, 1796,  271]),
 array([1.        , 1.        , 1.        , 0.99999994, 0.99999964,
        0.25848052, 0.2522796 , 0.24839874, 0.23939222, 0.23428331],
       dtype=float32))

In [72]:
movies_2.iloc[similar_movie[0]]

,movie_id,title,genre,year
3886,4004,the fast and the furious iv (2009),Adventure|Crime|Thriller,(2009)
3885,4003,the fast and the furious iii (2006),Adventure|Crime|Thriller,(2006)
3883,4001,the fast and the furious i (2001),Adventure|Crime|Thriller,(2001)
3884,4002,the fast and the furious ii (2003),Adventure|Crime|Thriller,(2003)
3887,4005,the fast and the furious v (2011),Adventure|Crime|Thriller,(2011)
2588,2657,"rocky horror picture show, the (1975)",Comedy|Horror|Musical|Sci-Fi,(1975)
3579,3648,"abominable snowman, the (1957)",Horror|Sci-Fi,(1957)
553,557,mamma roma (1962),Drama,(1962)
1796,1865,wild man blues (1998),Documentary,(1998)
271,274,man of the house (1995),Comedy,(1995)


내가 좋아하는 영화인 '분노의 질주 i'과 비슷한 영화로 분노의 질주 모든 시리즈가 잘 추천된 것을 볼 수 있음. 유사도 또한 1에 근접함.

그 외 영화는 유사도가 낮음. 확인해보니 분노의 질주와 겹치는 장르가 하나도 없음. 다른 유사한 작품이 없었던건가?

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [82]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data2[user], N=10, filter_already_liked_items=True)
movie_recommended

(array([2588,  287, 1260, 1028, 3617, 3801, 3090, 1159, 1323, 2325]),
 array([0.86886424, 0.73415995, 0.700181  , 0.67470455, 0.6538971 ,
        0.6530596 , 0.6421396 , 0.6420808 , 0.62988305, 0.6225684 ],
       dtype=float32))

In [83]:
movies_2.iloc[movie_recommended[0]]

,movie_id,title,genre,year
2588,2657,"rocky horror picture show, the (1975)",Comedy|Horror|Musical|Sci-Fi,(1975)
287,290,once were warriors (1994),Crime|Drama,(1994)
1260,1280,raise the red lantern (1991),Drama,(1991)
1028,1041,secrets & lies (1996),Drama,(1996)
3617,3686,flatliners (1990),Thriller,(1990)
3801,3871,shane (1953),Drama|Western,(1953)
3090,3159,fantasia 2000 (1999),Animation|Children's|Musical,(1999)
1159,1175,delicatessen (1991),Comedy|Sci-Fi,(1991)
1323,1344,cape fear (1962),Film-Noir|Thriller,(1962)
2325,2394,"prince of egypt, the (1998)",Animation|Musical,(1998)


__위 영화를 추천한 이유를 살펴보자__

In [85]:
# rocky horror picture show, the (1975)
movie_index = movie_to_idx[2657]
explain = als_model.explain(user, csr_data2, itemid=movie_index)

In [94]:
[(index_to_movie[i[0]], i[1]) for i in explain[1]]

[(4003, 0.17150233225752506),
 (4004, 0.17146720181771183),
 (4001, 0.1714393288428549),
 (4002, 0.1713611498795553),
 (4005, 0.171054763155315)]

4001부터 4005는 내가 추가한 분노의질주 시리즈의 movie id이다. 모든 시리즈가 비슷한 수준으로 "rocky horror picture show, the (1975)"의 추천에 기여함

In [100]:
# once were warriors (1994)
movie_index = movie_to_idx[290]
explain = als_model.explain(user, csr_data2, itemid=movie_index)

In [101]:
[(index_to_movie[i[0]], i[1]) for i in explain[1]]

[(4003, 0.14592340744211366),
 (4002, 0.14591440771365036),
 (4001, 0.1459078882919046),
 (4004, 0.14590228373833367),
 (4005, 0.1458604448879393)]

In [102]:
# prince of egypt, the (1998)
movie_index = movie_to_idx[2394]
explain = als_model.explain(user, csr_data2, itemid=movie_index)

In [103]:
[(index_to_movie[i[0]], i[1]) for i in explain[1]]

[(4002, 0.1258226457337937),
 (4003, 0.125777419917409),
 (4004, 0.1257670831188696),
 (4001, 0.125717423680168),
 (4005, 0.12542791962884822)]

위 추천된 모든 영화가 분노의질주 시리즈가 기여하여 추천됨을 알 수 있으나, 어떤 기준을 갖고 추천된건지는 알 수 없음.

## 회고

* 이번 프로젝트에서 사용된 Implicit learning은 영화 장르와 같은 명시적(explicit) 정보가 없는 경우에도 협업 필터링 기술(collaborative filtering techniques)을 통해 추천시스템이 작동함을 보여줌. 실제로 implicit 학습은 데이터가 적거나 명시적인 레이블이나 feautre가 부족한 상황에 특히 적합하다고 함.

* 영화 추천 시스템에서 협업 필터링은 user-item interaction인 영화 평점의 패턴을 활용하여 추천 근거를 생성함. 이때 근본적인 가정은 과거에 유사한 선호도를 가진 사용자가 미래에도 유사한 선호도를 가질 것임.

* 영화 추천 시스템의 Implicit learning 개요는 다음과 같음,
    1. user-item csr matrix: 영화 등급 데이터를 사용자 항목 매트릭스로 변환함. 여기서 행은 사용자를 나타내고 열은 영화를 나타내며 각 항목은 사용자가 영화에 부여한 등급에 해당됨.

    2. Matrix Factorization: ALS(Alternating Least Squares) 또는 SVD(Singular Value Decomposition)와 같은 행렬 분해 기술을 적용하여 사용자 항목 행렬을 더 낮은 차원의 잠재 요인으로 분해함.

    3. Latent Factor(잠재 요인) 표현: 잠재 요인은 사용자 및 영화의 기본 feature 또는 특성을 나타내는데, 해당 프로젝트에서는 rating(평점)이 사용됨. 이러한 요소에는 영화 장르와 같은 명시적인 특성이 없지만 데이터의 패턴과 관계를 캡처함으로써 영화 선호도를 파악하고 유사한 영화 추천, 사용자에게 맞는 영화 추천, 영화 추천에 기여한 다른 영화와 기여도 등을 파악할 수 있음.

    4. 유사성 계산: 잠재 요인을 기반으로 사용자 또는 항목 간의 유사성 메트릭을 계산함. 예를 들어 코사인 유사성 또는 유클리드 거리를 사용하여 ratings 패턴을 기반으로 사용자 간의 유사성을 측정할 수 있음.

    5. 추천 생성: 유사성 측정을 기반으로 유사한 사용자로부터 높은 평점을 받았거나 선호하는 항목을 식별하여 특정 사용자에게 영화를 추천함.

* Implicit learning을 처음 접해봐서 세 가지 데이터 셋(ratings, user, movie)을 모두 합쳐서 EDA와 feature engineering을 진행해야 하나 하는 혼란이 있었음. 암시적 학습은 명시적인 feature나 레이블이 아닌 데이터의 기본 패턴과 관계를 캡처하는 데 중점을 둔다는 점이 매우 새로웠음.